## Task 1: Binary Representations

In Task 1 there will be 4 functions used to showcase how a **32-bit number** can be manipulated.
These functions are mostly used in cryptography or low level programming which are all encryption methods.


### Rotating Bits Left (`rotl`)

The `rotl(x, n)` function is used to shift all bits by a number to the left by **n** places.
If a bit reaches the end of the 32 bit it wraps around and starts on the right side for it to continue.
**0xFFFFFFFF** is used to make sure we stay within the 32 bit range.
[Rotating bits of a number](https://www.geeksforgeeks.org/python3-program-to-rotate-bits-of-a-number/).

In [1]:
def rotl(x, n=1):
    """
    Rotates the bits in a 32-bit integer to the left by n places.
    """
    n = n % 32  # Ensure n is within valid bit range (0-31)
    return ((x << n) & 0xFFFFFFFF) | (x >> (32 - n))

#### **Rotating Bits Right (`rotr`)**

The `rotr(x, n)` function does the complete opposite to **rotl**.
It shifts all bits by a number to the righy by **n** places.
If a bit reaches the end of the 32 bit it wraps around and starts on the left side for it to continue.
Mainly used in cryptography because of how fast it is and is a good way to secure data.

In [2]:
def rotr(x, n=1):
    """
    Rotates the bits in a 32-bit integer to the right by n places.
    """
    n = n % 32  # Ensure n is within valid bit range (0-31)
    return (x >> n) | ((x << (32 - n)) & 0xFFFFFFFF)

#### **Bitwise Choice (`ch`)**

The `ch(x, y, z)` function uses x as a value and chooses between y and z.
If the bit in x is 1 it takes from y otherwise it takes from z.
This is used in SHA-256 hashing so it can decide to switch between values. 

In [3]:
def ch(x, y, z):
    """
    Chooses bits from y where x has bits set to 1, and from z where x has bits set to 0.
    Returns:
    int: Resulting 32-bit integer after bitwise choice
    """
    return (x & y) | (~x & z)  # If x bit is 1 -> choose from y, else from z


#### **Bitwise Majority (`maj`)**

The `maj(x, y, z)` function looks at x y and z and looks at each bit in them and then keeps the value that appears the most. 
The majority value is chosen if there a more than 1 chosen.
Is used in secure hashing algorithms when decisions need to be made with multiple values.

In [4]:
def maj(x, y, z):
    """
    Majority votes of bits in x, y, and z.

    Parameters:
    x (int): 32-bit integer
    y (int): 32-bit integer
    z (int): 32-bit integer

    Returns:
    int: Resulting 32-bit integer after bitwise majority
    """
    return (x & y) | (x & z) | (y & z)  # A bit is 1 if at least two of x, y, z have 1s

### Testing the Bitwise Functions

To show the functions being tested we define 3 **32-bit integers**

We use 3 **binary numbers** as inputs:
- `x = 0b10110011100011110000111100001111` → A randomly chosen **32-bit integer**.
- `y = 0b11001100110011001100110011001100` → A pattern of alternating bits.
- `z = 0b00001111000011110000111100001111` → High and low bit sequences.

These values showcase to us that the functions correctly handle the bits at different positions.

In [5]:
# Define 32-bit example values for testing
x = 0b10110011100011110000111100001111  # Example 32-bit integer
y = 0b11001100110011001100110011001100
z = 0b00001111000011110000111100001111

# Testing the functions
if __name__ == "__main__":
    print(f"Original x: {bin(x)}")
    print(f"rotl(x, 4): {bin(rotl(x, 4))}")
    print(f"rotr(x, 4): {bin(rotr(x, 4))}")
    print(f"ch(x, y, z): {bin(ch(x, y, z))}")
    print(f"maj(x, y, z): {bin(maj(x, y, z))}")

Original x: 0b10110011100011110000111100001111
rotl(x, 4): 0b111000111100001111000011111011
rotr(x, 4): 0b11111011001110001111000011110000
ch(x, y, z): 0b10001100100011000000110000001100
maj(x, y, z): 0b10001111100011110000111100001111


## Task 2: Hash Functions

In Task 2 we covert a string which in this instance is '(s)' into a numeric hash value.
Starts at `hashval = 0` which is the first value and then loops through each character.
The running total is then multiplied by 31 and adds the ASCII value of the character.
The result returned is stored in modulo 101. 

In [6]:
def hash_function(s: str) -> int:
    """
    Parameters:
    s (str): The input string.

    Returns:
    int: Hash value mod 101.
    """
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval
    return hashval % 101

### Testing the Hash Function

A list of words is used to check the output variety.
hash_function() is used on each word.
Results printed out.

In [7]:
# Testing the function
test_strings = ["john", "smith", "computational", "theory"]
for string in test_strings:
    print(f"Hash of '{string}': {hash_function(string)}")

Hash of 'john': 97
Hash of 'smith': 19
Hash of 'computational': 42
Hash of 'theory': 77


**Why Use 31 and 101?**

`31` is a prime number which is used alot in hashing as it spreads values and gives efficient bitwise operations. 
[Why does Java's `hashCode()` use 31 as a multiplier?](https://stackoverflow.com/questions/299304/why-does-javas-hashcode-in-string-use-31-as-a-multiplier) 
`101` is a prime number which is used as the modulo in this instance as it reduces the result and minimizes collisions in a small space. 

## Task 3: SHA256

In Task 3 we read the input file in binary mode so we can get the raw bytes which are stored in a variable.
The original message length is also stored in bits.

- The file is opened in **binary mode** (`rb`).
- The data is read and stored in `data`.
- The original message length is calculated in **bits**.

In [8]:
import os  # Importing OS for file handling

# Define file path
file_path = "test.txt"

try:
    with open(file_path, "rb") as f:
        data = f.read()  # Read file contents as bytes
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
    
# Store variables globally so they can be used in later cells
original_bit_length = len(data) * 8  

print("Step 1: File read successfully!")


Step 1: File read successfully!


SHA-256 padding is done by adding a single '1' bit after the message.
By doing this we ensure the message is beginning padding with a `10000000`** in binary.

- `b'\x80'` is added to the data which creates the padded message.
- This ensures padding starts with a `1` followed by zeros.

SHA-256 is a **cryptographic hash function** that follows the [NIST Secure Hash Standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf), ensuring secure and efficient message integrity.

In [9]:
# Append the '1' bit (0x80 in hex) to the message
padded_data = data + b'\x80'

print("Step 2: 1-bit appended.")

Step 2: 1-bit appended.


We then add zero bytes until the length of the message is **448 mod 512 bits**.

- A `while` loop continuously appends **zero bytes (`0x00`)**.
- It stops when the message length **mod 512** equals **448**.
- This makes sure that there is enough space for the final 64-bit length.

This step follows the SHA-256 padding rules as described in [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf).

In [10]:
# Add zero bytes until the length is 448 mod 512
while (len(padded_data) * 8) % 512 != 448:
    padded_data += b'\x00'  

print("Step 3: Padding with zeros done.")

Step 3: Padding with zeros done.


The original message length is then updated as a **64-bit big-endian integer**.  
The SHA-256 algorithm now knows how long the original message was before padding.

- We take the `original_bit_length` and **convert it to a 64-bit number**.
- This number is stored in **big-endian format** (`to_bytes(8, 'big')`).
- SHA-256 is then processed by the final 8 bytes to make sure it is correctly processed.

In [11]:
# Convert the original message length into an 8-byte (64-bit) big-endian integer
padded_data += original_bit_length.to_bytes(8, 'big')  

print("Step 4: Original length appended.")

Step 4: Original length appended.


The final message is then printed in **hex format**.
This shows that the padding is applied correctly.

- We print each byte of the padded message in **hexadecimal format**.

In [12]:
# Print the final padded message in hexadecimal format
print("SHA-256 Padding (Hex):")
print(" ".join(f"{byte:02x}" for byte in padded_data))

SHA-256 Padding (Hex):
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00


## Task 4: Prime Numbers

### Method 1: Trial Division (Brute Force)

**Trial Division method** checks each number to try and find out if that number is a prime number.

- Starting with **2** as it is the first prime number.
- Each number is checked to see if it is divisible by any of the previous primes.
- If the number is not divisible evenly it is then added to the list.

- This is brute force which is slow for large values as it has a **time complexity of \(O(n^2)\)**.

In [13]:
def trial_division_primes(n):
    """
    Finds the first 'n' prime numbers using the Trial Division method.
    """
    primes = []
    num = 2  # Start from the smallest prime number

    while len(primes) < n:
        is_prime = True
        for prime in primes:  
            if num % prime == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(num)
        num += 1  

    return primes

We now run the function which generates the first **1,000 prime numbers**.

- The function goes through each number and checks for primes.
- The first 10 primes found are then printed for verification.

In [14]:
trial_division_result = trial_division_primes(1000)

print(trial_division_result[:10])

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


**Advantages:**
- Simple and easy to understand.
- Works well with small values.

**Disadvantages:**
- Very slow for working with large values.
- Requires checking divisibility for every number.

### Method 2: Sieve of Eratosthenes (Efficient)

**Sieve of Eratosthenes method** is a faster way of finding primes as its main goal is to mark non-primes in a list.

- A boolean list is created where each index represents a number.
- Starting 2 and mark all multiples of each prime as non-prime.
- This is done until we have collected 1,000 primes.

- It has a **time complexity of \(O(n \log \log n)\)** which makes it much faster* than Trial Division.

In [15]:
def sieve_of_eratosthenes(n):
   
    limit = 10 * n  # Estimate upper limit
    sieve = [True] * limit  # True means "assumed prime"
    sieve[0] = sieve[1] = False  # 0 and 1 are not primes

    for i in range(2, int(limit**0.5) + 1):
        if sieve[i]:  # If i is prime
            for multiple in range(i * i, limit, i):
                sieve[multiple] = False  # Mark multiples as non-prime

    primes = [num for num, is_prime in enumerate(sieve) if is_prime][:n]
    return primes

We not run the function to generate the first 1,000 prime numbers.

- We run the Sieve of Eratosthenes function.
- The first 10 primes are printed for verification.

In [16]:
# Generate first 1000 primes using Sieve of Eratosthenes
sieve_result = sieve_of_eratosthenes(1000)

# Print first 10 primes for verification
print(sieve_result[:10])

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


 **Advantages:**
- A lot faster than Trial Division.
- Large values work well.

 **Disadvantages:**
- Requires extra memory to store the boolean array.

## Trial Division vs. Sieve of Eratosthenes

| Algorithm               | Time Complexity      | Space Complexity | Best Used For |
|-------------------------|---------------------|------------------|---------------|
| Trial Division          | \(O(n^2)\)         | \(O(1)\)         | Small numbers |
| Sieve of Eratosthenes   | \(O(n \log \log n)\) | \(O(n)\)         | Large numbers |


- **Trial Division** is more simple but not as effective for large values.
- **Sieve of Eratosthenes** is a lot  faster but does require extra memory.

## Task 5: Roots

## Compute the first 100 prime numbers

There needs to be 100 prime numbers before we start to calculate square roots.

- **Sieve of Eratosthenes** is used to generate the first 100 prime numbers as it is the most efficient method.
- By doing this we make sure we don’t manually list primes.

In [17]:
from math import sqrt

def sieve_of_eratosthenes(n):
    """
    Finds the first 'n' prime numbers using the Sieve of Eratosthenes.
    """
    limit = 1000  # Estimate a safe upper limit
    sieve = [True] * limit  # True means "assumed prime"
    sieve[0] = sieve[1] = False  # 0 and 1 are not primes

    for i in range(2, int(limit**0.5) + 1):
        if sieve[i]:  # If i is prime
            for multiple in range(i * i, limit, i):
                sieve[multiple] = False  # Mark multiples as non-prime

    primes = [num for num, is_prime in enumerate(sieve) if is_prime][:n]
    return primes

#  Generate first 100 prime numbers
first_100_primes = sieve_of_eratosthenes(100)
print(first_100_primes[:10])  # Print first 10 primes for verification


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


## Compute square roots & extract fractional part

- We calculate the square root of each prime.
- We isolate the fractional part.
- The fractional part is then converted to binary.

In [18]:
def get_fractional_part(number):
    """
    Extracts the fractional part of a number.
    """
    return number - int(number)  # Remove integer part

# Example: Extract fractional part of sqrt(2)
example_sqrt = sqrt(2)
fractional_part = get_fractional_part(example_sqrt)
print(f"Example: sqrt(2) = {example_sqrt}, Fractional Part = {fractional_part}")

Example: sqrt(2) = 1.4142135623730951, Fractional Part = 0.41421356237309515


## Converting the fractional part to 32-bit binary

- We multiply the fractional part by 2 repeatedly.
- The integer part of each multiplication then gives us 1s and 0s.
- We extract 32 bits from this conversion.

In [19]:
def fractional_to_binary(fraction, bits=32):
    """
    Converts a fractional part to a 32-bit binary representation.
    """
    binary_str = ""
    for _ in range(bits):
        fraction *= 2
        if fraction >= 1:
            binary_str += "1"
            fraction -= 1
        else:
            binary_str += "0"
    return binary_str

binary_fractional_part = fractional_to_binary(fractional_part)
print(f"32-bit binary of sqrt(2) fractional part: {binary_fractional_part}")

32-bit binary of sqrt(2) fractional part: 01101010000010011110011001100111


## Computing and displaying results for the first 100 Primes

- For each prime number we must compute the square root.
- Extract the fractional then convert it to binary.
- The print out the first 10 results for verification.

In [20]:
# Compute the first 32 bits of the fractional part of sqrt(primes)
results = {}

for prime in first_100_primes:
    sqrt_value = sqrt(prime)  # Compute square root
    fractional_part = get_fractional_part(sqrt_value)  # Extract fractional part
    binary_representation = fractional_to_binary(fractional_part)  # Convert to binary
    results[prime] = binary_representation

# Print first 10 results
for prime, binary in list(results.items())[:10]:
    print(f"Prime: {prime}, Binary (32 bits): {binary}")

Prime: 2, Binary (32 bits): 01101010000010011110011001100111
Prime: 3, Binary (32 bits): 10111011011001111010111010000101
Prime: 5, Binary (32 bits): 00111100011011101111001101110010
Prime: 7, Binary (32 bits): 10100101010011111111010100111010
Prime: 11, Binary (32 bits): 01010001000011100101001001111111
Prime: 13, Binary (32 bits): 10011011000001010110100010001100
Prime: 17, Binary (32 bits): 00011111100000111101100110101011
Prime: 19, Binary (32 bits): 01011011111000001100110100011001
Prime: 23, Binary (32 bits): 11001011101110111001110101011101
Prime: 29, Binary (32 bits): 01100010100110100010100100101010


## Task 6: Proof of Work

## Load a Dictionary Word List

In this task we will use a list of English words which can be gathered in /usr/share/dict/words which is a local file or use the fallback method which is a word list.
By doing this we ensure that the words we use are from a dictionary.

In [21]:
try:
    with open("/usr/share/dict/words") as f:
        words = [line.strip() for line in f if line.strip().isalpha()]
except FileNotFoundError:
    # Fallback list
    words = ["apple", "banana", "orange", "proof", "of", "work", "zero", "hash", "challenge"]

## For each word we compute a SHA-256 Hash. 

For each word we:
1. Have compute the SHA-256 hash.
2. Convert the hash to binary.
3. At the beginning we count how many 0 bits there are

In [22]:
import hashlib

def count_leading_zero_bits(hex_digest):
    """Convert hex digest to binary and count leading zero bits."""
    bin_digest = bin(int(hex_digest, 16))[2:].zfill(256) 
    return len(bin_digest) - len(bin_digest.lstrip("0"))

## Find the Word or words with the most leading 0 bits

To do this we have to track the maximum number of leading zeros and then we have to store the words that match it.

In [23]:
max_zeros = 0
best_words = []

for word in words:
    h = hashlib.sha256(word.encode()).hexdigest()
    zeros = count_leading_zero_bits(h)

    if zeros > max_zeros:
        max_zeros = zeros
        best_words = [(word, h, zeros)]
    elif zeros == max_zeros:
        best_words.append((word, h, zeros))

# Print results
print(f"Max leading zeros: {max_zeros}")
for w, h, z in best_words:
    print(f"Word: {w}, Zeros: {z}, SHA256: {h}")

Max leading zeros: 8
Word: work, Zeros: 8, SHA256: 00e13ed7af55b27622f1d6eab5bec0147e68efe28dc2b12461117afa1a5ed40e


## Checking for Word Validity

We now have to make sure the word that was found is an English word. 
We do this by:

- By using /usr/share/dict/words to see if it exits.
- Use PyEnchant to check.
- Or using a dictionary API.

## Task 7: Turing Machines

## Turing Machine 

In this task we created a simple Turing Machine which adds 1 to a binary number.

This is done by:

- Binary number is written on a tape (e.g. 100111)
- The bit that is most to the right is the least significant bit (LSB)
- The turning machine then does carry logic which is like how binary addition works manually.

- 1 is only written when it sees a 0.
- If a 1 is seen then it writes a 0 and moves left to carry on.
- If there is a blank reached which is at the start it then write a 1 at the front.

## State Transition Table

| Current State | Read Symbol | Write Symbol | Move | Next State |
|---------------|-------------|--------------|------|------------|
| `q0`          | `1`         | `0`          | L    | `q0`       |
| `q0`          | `0`         | `1`          | R    | `HALT`     |
| `q0`          | `B`         | `1`          | R    | `HALT`     |

**Explanation**:
- The main state is q0 which is where the machine does the carry logic
q0 = main carry-propagation state
- When there is a 1 read it then writes 0 and moves on to the left
- If there is a 0 seen then the carry is ended and a 1 is then written and it stops
- When the machine sees a blank which is a (B) that means the number was at 1 so it then writes a new 1 at the start

In [24]:
def turing_add_one(tape_input):
    # Convert input string to list (tape), and add blank symbol at the beginning
    tape = ['B'] + list(tape_input)
    head = len(tape) - 1  # Start at right-most symbol (LSB)
    state = 'q0'

    while state != 'HALT':
        symbol = tape[head]

        if state == 'q0':
            if symbol == '1':
                tape[head] = '0'
                head -= 1
            elif symbol == '0':
                tape[head] = '1'
                state = 'HALT'
            elif symbol == 'B':
                tape[head] = '1'
                state = 'HALT'

    # Remove leading blank if not needed
    result = ''.join(tape).lstrip('B')
    return result

# Test case
input_tape = '100111'
output_tape = turing_add_one(input_tape)

print(f"Input:  {input_tape}")
print(f"Output: {output_tape}")

Input:  100111
Output: 101000


## Task 8: Computational Complexity

### Bubble Sort with Comparison Counting
Created a function that does a normal bubble sort and after every time it compares 2 elements it then counts that comparison.

In [25]:
def bubble_sort_with_count(arr):
    """
    Sorts the list using Bubble Sort and counts the comparisons made.
    """
    n = len(arr)
    count = 0
    a = arr.copy()  # To avoid changing the original permutation
    for i in range(n):
        for j in range(0, n - i - 1):
            count += 1  # Each comparison is counted
            if a[j] > a[j + 1]:
                a[j], a[j + 1] = a[j + 1], a[j]
    return count

### Generate All Permutations

Created all 120 permutations of [1, 2, 3, 4, 5] using Python's itertools.permutations.
Used the bubble sort function for every permutation and displayed the outcome.

In [27]:
from itertools import permutations

L = [1, 2, 3, 4, 5]
all_perms = list(permutations(L))

for perm in all_perms:
    comparison_count = bubble_sort_with_count(list(perm))
    print(f"Permutation: {perm}, Comparisons: {comparison_count}")

Permutation: (1, 2, 3, 4, 5), Comparisons: 10
Permutation: (1, 2, 3, 5, 4), Comparisons: 10
Permutation: (1, 2, 4, 3, 5), Comparisons: 10
Permutation: (1, 2, 4, 5, 3), Comparisons: 10
Permutation: (1, 2, 5, 3, 4), Comparisons: 10
Permutation: (1, 2, 5, 4, 3), Comparisons: 10
Permutation: (1, 3, 2, 4, 5), Comparisons: 10
Permutation: (1, 3, 2, 5, 4), Comparisons: 10
Permutation: (1, 3, 4, 2, 5), Comparisons: 10
Permutation: (1, 3, 4, 5, 2), Comparisons: 10
Permutation: (1, 3, 5, 2, 4), Comparisons: 10
Permutation: (1, 3, 5, 4, 2), Comparisons: 10
Permutation: (1, 4, 2, 3, 5), Comparisons: 10
Permutation: (1, 4, 2, 5, 3), Comparisons: 10
Permutation: (1, 4, 3, 2, 5), Comparisons: 10
Permutation: (1, 4, 3, 5, 2), Comparisons: 10
Permutation: (1, 4, 5, 2, 3), Comparisons: 10
Permutation: (1, 4, 5, 3, 2), Comparisons: 10
Permutation: (1, 5, 2, 3, 4), Comparisons: 10
Permutation: (1, 5, 2, 4, 3), Comparisons: 10
Permutation: (1, 5, 3, 2, 4), Comparisons: 10
Permutation: (1, 5, 3, 4, 2), Comp